# TUTORIAL W36

In this tutorial, we will get acquainted with working with MEG data on UCloud  
We will be using the MNE-sample-data that contains visual and auditory stimulations  
The dataset also contains anatomical MR-data processed with FreeSurfer, meaning that we have access to a forward model linking the source space to sensor space.  
With that we can create an inverse model for reconstructing the sensor space visual and auditory responses in the source space.

**Make sure that you have followed the instructions for setting up your environment, local and UCloud before beginning on this.** The instructions can be found here: https://github.com/ualsbombe/2025_advanced_cognitive_neuroscience/blob/main/README.md



In [ ]:
## IMPORTS

import mne ## MNE-Python for analysing data
## below magic provides interactive plots in notebook
%matplotlib widget
from os import chdir
from os.path import join
import matplotlib.pyplot as plt ## for basic plotting
import matplotlib as mpl ## for setting default parameters

In [ ]:
#%% SET DEFAULT PLOTTING PARAMETERS

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['font.size'] = 24
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['lines.linewidth'] = 3

## SAMPLE DATA SET (https://mne.tools/stable/documentation/datasets.html#sample)
*These data were acquired with the Neuromag Vectorview system at MGH/HMS/MIT Athinoula A. Martinos Center Biomedical Imaging. EEG data from a 60-channel electrode cap was acquired simultaneously with the MEG. The original MRI data set was acquired with a Siemens 1.5 T Sonata scanner using an MPRAGE sequence.*

*In this experiment, checkerboard patterns were presented to the subject into the left and right visual field, interspersed by tones to the left or right ear. The interval between the stimuli was 750 ms. Occasionally a smiley face was presented at the center of the visual field. The subject was asked to press a key with the right index finger as soon as possible after the appearance of the face.*

Change the path to your relevant path below


In [ ]:
#%% LOAD SAMPLE DATA SET

sample_path = '/work/MEG_data/MNE-sample-data' ## UCloud
sample_path = '/home/lau/mne_data/MNE-sample-data/' ## local
sample_meg_path = join(sample_path, 'MEG', 'sample')
chdir(sample_meg_path)
subjects_dir = '../../subjects/'

In [ ]:
#%% READ RAW

raw_sample = mne.io.read_raw_fif('sample_audvis_raw.fif', preload=True)

In [ ]:
## PLOT RAW DATA
fig = raw_sample.plot()

In [ ]:
## COMPUTE POWER SPECTRAL DENSITY
psd = raw_sample.compute_psd() 

In [ ]:
## PLOT POWER SPECTRAL DENSITY
fig = psd.plot() # online low-pass filter of 170 Hz

In [ ]:
## FILTERING DATA WITH LOW-PASS FILTER
raw_sample.filter(l_freq=None, h_freq=40) # low-pass filter of 40 Hz

In [ ]:
## PLOTTING POWER SPECTRAL DENSITY AFTER LOW-PASS FILTER
fig = raw_sample.compute_psd().plot()

In [ ]:
#%% FIND EVENTS

events_sample = mne.find_events(raw_sample)
mne.viz.plot_events(events_sample)
mne.viz.plot_events(events_sample, sfreq=raw_sample.info['sfreq']);

## LA: 1: Response to left-ear auditory stimulus (a tone)
## RA: 2: Response to right-ear auditory stimulus
## LV: 3: Response to left visual field stimulus (checkerboard)
## RV: 4: Response to right visual field stimulus
## smiley: 5: Response to the smiley face
## button: 32: Response triggered by the button press
# https://mne.tools/stable/overview/datasets_index.html#sample

In [ ]:
#%% EPOCH THE DATA
event_id = dict(LA=1) ## we'll just look at this one event
tmin = -0.200 # s
tmax =  0.600 # s
baseline = (None, 0) # s (from beginning to 0); for demeaning

## segment the data
epochs_sample = mne.Epochs(raw_sample, events_sample,
                           event_id, tmin, tmax, baseline, preload=True)
epochs_sample.set_eeg_reference(projection=True);

In [ ]:
## PLOT EPOCHS
fig = epochs_sample.plot()

In [ ]:
#%% AVERAGE (EVOKED)
evoked_sample = epochs_sample.average()
print(evoked_sample)


In [ ]:
## PLOTTING AVERAGES
evoked_sample.plot() 
mne.viz.plot_evoked_topo(evoked_sample);

In [ ]:
#%% HAVING MORE THAN ONE EVENT

event_id = dict(LA=1, RA=2, LV=3, RV=4)
more_epochs = mne.Epochs(raw_sample, events_sample,
                           event_id, tmin=-0.200, tmax=0.600,
                           baseline=(None, 0),
                           preload=True)

more_evokeds = list()
for event in more_epochs.event_id:
    more_evokeds.append(more_epochs[event].average());



In [ ]:
## PLOT SEVERAL EVENTS
mne.viz.plot_evoked_topo(more_evokeds);

In [ ]:
## READ FORWARD MODEL
fwd = mne.read_forward_solution('sample_audvis-meg-eeg-oct-6-fwd.fif')

In [ ]:
## PLOT SOURCE SPACE

# extract source space
src = fwd['src']
##plot alignment does not work on UCloud
mne.viz.plot_alignment(info=epochs_sample.info, src=src, subjects_dir='../../subjects/',
                       subject="sample", surfaces="white", trans='sample_audvis_raw-trans.fif')

## 2D alternative

#mne.viz.plot_sensors(raw_sample.info);

In [ ]:
## INFO ABOUT WHERE CHANNELS ARE (in subject's head space)
info = epochs_sample.info
print(info['chs'][2]['loc'])

In [ ]:
## INFO ABOUT WHERE SOURCE ARE (in subject's MR space)
src = fwd['src']
# src = mne.source_space.setup_source_space(subject='sample', subjects_dir=subjects_dir)
print(fwd['src'][0]['rr'])

In [ ]:
## TRANSFORMATION MATRIX THAT MAPS THE TWO SPACES ONTO ANOTHER
trans = mne.read_trans('sample_audvis_raw-trans.fif')
print(trans)

In [ ]:
## BOUNDARY ELEMENT METHOD

## describe the surfaces and their conductivities
#bem_model = mne.bem.make_bem_model(subject=subject, subjects_dir=subjects_dir,
#                           conductivity=[0.3, 0.006, 0.3]) ## three layer model

## model how electrical potentials spread to the electrodes and how the 
# currents of the brain are related to the magnetic field measured at the
# sensors
#bem_solution = mne.bem.make_bem_solution(bem_model)

bem_solution = mne.bem.read_bem_solution('../../subjects/sample/bem/sample-5120-5120-5120-bem-sol.fif')

print(bem_solution['solution'])

In [ ]:
## create a forward solution using four ingredients; L(r),
# which maps how sources in the brain link to sensors in the helmet:
# info: information about channel positions and sensor types
# trans: the transformation needed to align channel positions with MR
# src: the source model (in this case, a cortical sheet)
# bem_solution: modelling of how electrical currents spread toward the
#               electrodes   

#fwd = mne.make_forward_solution(info=info, trans=trans, src=src, bem=bem_solution)

In [ ]:
## again used to whiten the data, i.e. normalizing magnetometers, gradiometers
## and electrode readings to make them comparable
noise_cov = mne.compute_covariance(epochs_sample, tmin=None, tmax=0)
noise_cov.plot(raw_sample.info)

In [ ]:
inverse_operator = mne.minimum_norm.make_inverse_operator(epochs_sample.info, fwd,
                                                          noise_cov)

# estimating the source pattern for each time point Vvox(t)
# right auditory stimulus (evokeds_sample[1])
MNE = mne.minimum_norm.apply_inverse(evoked_sample, inverse_operator,
                                     method='MNE')
## standard is to use method=dSPM for depth correction

In [ ]:
MNE.plot(subject='sample', subjects_dir=subjects_dir, hemi='both')